#### Exhaustive Feature Selection

Exhaustive Feature Selection finds the best subset of features of all possible feature subsets, according to a determined performance metric for a certain machine learning algorithm.

For example, if we train a logistic regression and the dataset consists of 4 features, the algorithm will evaluate all 15 feature combinations as follows:

all possible combinations of 1 feature
all possible combinations of 2 features
all possible combinations of 3 features
all the 4 features
and select the one that results in the best performance (e.g., classification accuracy) of the logistic regression.

Exhaustive Feature Selection is a greedy algorithm as it evaluates all possible feature combinations. It is very computationally expensive, and sometimes, if feature space is big, even unfeasible.

There is a special package for python that implements this type of feature selection: [mlxtend](http://rasbt.github.io/mlxtend/)

In the mlxtend implementation of the Exhaustive Feature Selection, the stopping criteria is an arbitrarily set number of features. So the search will finish when we reach the desired number of selected features.

This is somewhat arbitrary, we might be selecting a sub-optimal number of features, or likewise, a high number of features. But, by looking at the performance metric returned by the algorithm as it selects the features, we can have a view, if more features do add value, or not.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score, r2_score

from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS

**classification**

In [2]:
# load dataset

# few rows to speed things up

data = pd.read_csv('../datasets/dataset_2.csv', nrows=10000)
data.shape

(10000, 109)

In [3]:
# separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['target'], axis=1),
    data['target'],
    test_size=0.3,
    random_state=0
)

X_train.shape, X_test.shape

((7000, 108), (3000, 108))

**Remove correlated features**

he Exhaustive Feature Selection takes a long time to run.

In [4]:
# remove correlated features to reduce the feature space

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

# note that we reduce the correlation threshold
# to remove more features
corr_features = correlation(X_train, 0.6)
print('correlated features: ', len(set(corr_features)) )

correlated features:  59


In [5]:
# removed correlated features
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((7000, 49), (3000, 49))

**Exhaustive Feature Selection**

In [6]:
##################################

# in order to shorter search time for the demonstration
# i will ask the algorithm to try all possible 1 and 2
# feature combinations

# if you have access to a multicore or distributed computer
# system you can try more greedy searches

###################################

# within the EFS we indicate:

# 1) the algorithm we want to create, in this case RandomForests
# (note that I use few trees to speed things up)

# 2) the number of minimum features we want our model to have

# 3) the number of maximum features we want our model to have

# with 2 and 3 we regulate the number of possible feature combinations to
# be evaluated by the model.

# 4) the evaluation metric: in this case the roc_auc
# 5) the cross-validation

# this is going to take a while, do not despair

efs = EFS(RandomForestClassifier(n_estimators=5,
                                 n_jobs=4,
                                 random_state=0,
                                 max_depth=2),
          min_features=1,
          max_features=2,
          scoring='roc_auc',
          print_progress=True,
          cv=2)

# search features
efs = efs.fit(np.array(X_train), y_train)

Features: 1225/1225

The search evaluated 1225 feature combinations.

In [7]:
efs.best_idx_

(12, 32)

In [8]:
selected_feat = X_train.columns[list(efs.best_idx_)]
selected_feat

Index(['var_16', 'var_55'], dtype='object')

**Compare performance of feature subsets**

In [9]:
# function to train random forests and evaluate the performance

def run_randomForests(X_train, X_test, y_train, y_test):
    
    rf = RandomForestClassifier(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)

    print('Train set')
    pred = rf.predict_proba(X_train)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
    
    print('Test set')
    pred = rf.predict_proba(X_test)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

In [10]:
# evaluate performance of classifier using selected features

run_randomForests(X_train[selected_feat],
                  X_test[selected_feat],
                  y_train, y_test)

Train set
Random Forests roc-auc: 0.7152616074556793
Test set
Random Forests roc-auc: 0.6868327155531925


In [11]:
# and for comparison, we train random forests using
# all features (except the correlated ones, which we removed already)

run_randomForests(X_train,
                  X_test,
                  y_train, y_test)

Train set
Random Forests roc-auc: 0.7509464508443697
Test set
Random Forests roc-auc: 0.6874840935262856


**Regression**

In [12]:
# load dataset
data = pd.read_csv('../datasets/houseprice.csv')
data.shape

(1460, 81)

In [13]:
# In practice, feature selection should be done after data pre-processing,
# so ideally, all the categorical variables are encoded into numbers,
# and then you can assess how deterministic they are of the target

# here for simplicity I will use only numerical variables
# select numerical columns:

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(data.select_dtypes(include=numerics).columns)
data = data[numerical_vars]
data.shape

(1460, 38)

In [14]:
# separate train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['SalePrice'], axis=1),
    data['SalePrice'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((1022, 37), (438, 37))

**Remove correlated features**

In [15]:
# find and remove correlated features

def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

# note thta we reduce the threshold to remove more features
corr_features = correlation(X_train, 0.6)
print('correlated features: ', len(set(corr_features)) )

correlated features:  9


In [16]:
# removed correlated  features
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((1022, 28), (438, 28))

In [17]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

**Exhaustive Feature Selection**

In [18]:
# exhaustive search

# in order to shorter search time for the demonstration
# i will ask the algorithm to try all possible 10 and 11
# feature combinations

# if you have access to a multicore or distributed computer
# system you can try more greedy searches

efs = EFS(RandomForestRegressor(n_estimators=5,
                                n_jobs=4,
                                random_state=0,
                                max_depth=2),
          min_features=1,
          max_features=2,
          scoring='r2',
          print_progress=True,
          cv=2)

efs = efs.fit(np.array(X_train), y_train)

Features: 406/406

The searched evaluated 406 feature combinations.

In [19]:
efs.best_idx_

(4, 9)

In [21]:
X_train.columns[list(efs.best_idx_)]

Index(['OverallQual', 'BsmtFinSF1'], dtype='object')

**Compare performance of feature subsets**

In [22]:
# function to train random forests and evaluate the performance

def run_randomForests(X_train, X_test, y_train, y_test):
    
    rf = RandomForestRegressor(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)

    print('Train set')
    pred = rf.predict(X_train)
    print('Random Forests roc-auc: {}'.format(r2_score(y_train, pred)))
    
    print('Test set')
    pred = rf.predict(X_test)
    print('Random Forests roc-auc: {}'.format(r2_score(y_test, pred)))

In [23]:
selected_feat = X_train.columns[list(efs.best_idx_)]

In [24]:
# evaluate performance of algorithm built
# using selected features

run_randomForests(X_train[selected_feat],
                  X_test[selected_feat],
                  y_train, y_test)

Train set
Random Forests roc-auc: 0.7598962603686706
Test set
Random Forests roc-auc: 0.6856450799849569


In [25]:
# and for comparison, we train random forests using
# all features (except the correlated ones, which we removed already)

run_randomForests(X_train,
                  X_test,
                  y_train, y_test)

Train set
Random Forests roc-auc: 0.8454047044087043
Test set
Random Forests roc-auc: 0.7806708079073277


In this case, the performance decreased quite a lot, so there are additional features that are good predictors of the target.